# Complex TFA multiple Inversions

This notebook performs the inversion using Levenberg-Marquadt's algorithm of total field anomaly (TFA).

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import cPickle as pickle
import os

from fatiando.vis import mpl

/home/leonardo/anaconda2/lib/python2.7/site-packages/fatiando/vis/mpl.py:76: UserWarning: This module will be removed in v0.6. We recommend the use of matplotlib.pyplot module directly. Some of the fatiando specific functions will remain.
  "specific functions will remain.")


In [2]:
from datetime import date
today = date.today()
# dd/mm/YY
d4 = today.strftime("%b-%d-%Y")

### Auxiliary functions

In [3]:
import sys
sys.path.insert(0, '../../code')

import mag_polyprism_functions as mfun

# Input

### Importing model parameters

In [4]:
model_dir = 'data/model_real_obs_pts.pickle'
data_dir = 'data/real_obs_pts/data.pickle'
#grid_dir = '../grids/flightlines_grid_2100pts_100km2_-150m.pickle'

In [5]:
with open(model_dir) as w:
        model = pickle.load(w)

In [6]:
with open(data_dir) as w:
        data = pickle.load(w)

In [7]:
dobs = data['tfa_obs']
xp = data['grid'][0]
yp = data['grid'][1]
zp = data['grid'][2]
N = data['grid'][3]

### Parameters of the initial model

In [8]:
M = 20 # number of vertices per prism
L = 10 # number of prisms
P = L*(M+2) + 1 # number of parameters

# magnetization direction
incs = model['inc']
decs = model['dec']

int_min = 9.
int_max = 15.
intensity = np.linspace(int_min, int_max, 6)

# depth to the top, thickness and radius
z0_min = -400.
z0_max = -200.
z0 = np.linspace(z0_min, z0_max, 6)
dz = 650.
r = 800.
x0 = -300.
y0 = 300.

# main field
inc, dec = data['main_field']

In [9]:
z0

array([-400., -360., -320., -280., -240., -200.])

In [10]:
intensity

array([ 9. , 10.2, 11.4, 12.6, 13.8, 15. ])

### Limits

In [11]:
# limits for parameters in meters
rmin = 10.
rmax = 3000.
x0min = -4000.
x0max = 4000.
y0min = -4000.
y0max = 400.
dzmin = 200.
dzmax = 1000.

mmin, mmax = mfun.build_range_param(M, L, rmin, rmax, x0min, x0max, y0min, y0max, dzmin, dzmax)

### Variation

In [12]:
# variation for derivatives
deltax = 0.01*np.max(1000.)
deltay = 0.01*np.max(1000.)
deltar = 0.01*np.max(1000.)
deltaz = 0.01*np.max(100.)

### Outcropping parameters

In [13]:
# outcropping body parameters
m_out = np.zeros(M + 2)
#m_out = model['param_vec'][:M+2]

### Folder to save the results

In [14]:
mypath = 'data/real_obs_pts/tfa_inversion/multiple_int_min%d_int_max%d_z0_min%d_z0_max%d_r%d_dz%d_V%d_L%d' % (int_min, int_max, z0_min, z0_max, r, dz, M, L)
mypath += d4
if not os.path.isdir(mypath):
   os.makedirs(mypath)

In [15]:
mypath

'data/real_obs_pts/tfa_inversion/multiple_int_min9_int_max15_z0_min-400_z0_max-200_r800_dz650_V20_L10Feb-15-2020'

In [16]:
# output of inversion
inversion = dict()

### Regularization parameters

In [17]:
#lamb = th*0.01 # Marquadt's parameter
lamb = 10.0
dlamb = 10.      # step for Marquadt's parameter

a1 = 1.0e-5  # adjacent radial distances within each prism
a2 = 1.0e-4   # vertically adjacent radial distances
a3 = 0.     # outcropping cross-section
a4 = 0.     # outcropping origin
a5 = 1.0e-4    # vertically adjacent origins
a6 = 1.0e-7   # zero order Tikhonov on adjacent radial distances
a7 = 1.0e-5     # zero order Tikhonov on thickness of each prism

In [18]:
delta = np.array([deltax, deltay, deltar, deltaz])
alpha = np.array([a1, a2, a3, a4, a5, a6, a7])

In [19]:
itmax = 30
itmax_marq = 10
tol = 1.0e-4     # stop criterion

In [20]:
inversion['x'] = xp
inversion['y'] = yp
inversion['z'] = zp
inversion['observed_data'] = dobs

In [21]:
inversion['inc_dec'] = [incs, decs]
inversion['z0'] = z0
inversion['initial_dz'] = dz
inversion['intial_r'] = r
#inversion['initial_estimate'] = model0
#inversion['initial_data'] = d0
inversion['limits'] = [rmin, rmax, x0min, x0max, y0min, y0max, dzmin, dzmax]
inversion['regularization'] = alpha
inversion['tol'] = tol
inversion['main_field'] = [inc, dec]
inversion['intensity'] = intensity

### Inversion

In [22]:
inversion_results = []
for j, z in enumerate(z0):
    for k, i in enumerate(intensity):
        alpha = np.array([a1, a2, a3, a4, a5, a6, a7])
        print 'inversion: %d  top: %d  intensity: %d' % (j*z0.size + k, z, i)
        model0, m0 = mfun.initial_cylinder(M, L, x0, y0, z, dz, r, inc, dec, incs, decs, i)
        d_fit, m_est, model_est, phi_list, model_list, res_list = mfun.levmarq_tf(
            xp, yp, zp, m0, M, L, delta,
            itmax, itmax_marq, lamb,
            dlamb, tol, mmin, mmax,
            m_out, dobs, inc, dec,
            model0[0].props, alpha, z, dz
        )
        inversion_results.append([d_fit, m_est, model_est, phi_list, model_list, dobs - d_fit, res_list])

inversion: 0  top: -400  intensity: 9
it:  0   it_marq:  0   lambda: 1e+01   init obj.: 2.31802e+04  fin obj.: 1.52926e+04
it:  1   it_marq:  0   lambda: 1e+00   init obj.: 1.52926e+04  fin obj.: 7.17496e+03
it:  2   it_marq:  0   lambda: 1e-01   init obj.: 7.17496e+03  fin obj.: 6.11790e+03
it:  3   it_marq:  0   lambda: 1e-02   init obj.: 6.11790e+03  fin obj.: 9.09161e+03
it:  3   it_marq:  1   lambda: 1e-01   init obj.: 6.11790e+03  fin obj.: 1.63454e+04
it:  3   it_marq:  2   lambda: 1e+00   init obj.: 6.11790e+03  fin obj.: 1.74506e+04
it:  3   it_marq:  3   lambda: 1e+01   init obj.: 6.11790e+03  fin obj.: 5.35080e+03
it:  4   it_marq:  0   lambda: 1e+00   init obj.: 5.35080e+03  fin obj.: 2.06152e+03
it:  5   it_marq:  0   lambda: 1e-01   init obj.: 2.06152e+03  fin obj.: 5.79435e+03
it:  5   it_marq:  1   lambda: 1e+00   init obj.: 2.06152e+03  fin obj.: 1.41243e+03
it:  6   it_marq:  0   lambda: 1e-01   init obj.: 1.41243e+03  fin obj.: 1.18021e+03
it:  7   it_marq:  0   lamb

# Results

In [23]:
inversion['results'] = inversion_results

In [24]:
file_name = mypath+'/inversion.pickle'
with open(file_name, 'w') as f:
    pickle.dump(inversion, f)